In [11]:
import customtkinter # used for graphical representation
import time          # used to enhance the effect
import colorsys      # handeling colorformat

customtkinter.set_appearance_mode("dark")

In [12]:
class InfoWindow(customtkinter.CTkToplevel):
    def __init__(self, parent):
        super().__init__(parent)

        self.attributes('-fullscreen',True)
        self.title("Description")
        self.parent = parent

        self.back = customtkinter.CTkButton(self,
                text='Back',
                fg_color=parent.uiColor,
                hover_color=parent.hoverColor,
                command=self.backCallback)

        self.back.place(relx=.5,rely=.5,anchor=customtkinter.CENTER)
    def backCallback(self):
        self.parent.deiconify()
        self.destroy()

In [13]:
class WarningWindow(customtkinter.CTkToplevel):
    def __init__(self, parent):
        super().__init__(parent)
        self.attributes('-fullscreen',True)
        self.title("Epilepsiewarnung")

        self.check_var = "off"
        self.parent = parent
        self.text="Sie sind im Begriff eine Funktion aufzurufen, welche die Hintergrundfarbe des \nBildschirms in regelmäßigen Zeitabständen (0.5 bis 2sek) von schwarz zu weiß wechseln lässt.\n\nManche Menschen zeigen bei bestimmten Arten von flimmerndern Lichtquellen oder ähnlichen,in der Umgebung täglich vorkommenden Elementen eine \nNeigung zu epileptischen Krisen oder Ohnmachtsanfällen.\n\nDieses Risiko kann bei bestimmten Fernsehbildern oder Videospielen auftreten. Das Phänomen kann auch auftreten, ohne dass die betreffende Person in diesem Zusammenhang je medizinisch behandelt wurde oder einen epileptischen \nAnfall erlitten hat. \n\nSollten Sie selbst oder ein Mitglied Ihrer Familie je bei flimmernden Lichtquellen epilepsie-ähnliche Symptome empfunden haben, so verzichten Sie bitte auf \ndie Nutzung dieses Features.\nBrechen Sie das Programm sofort ab, und konsultieren Sie einen Arzt, falls Sie \neines der folgenden Symptome verspüren: Schwindel, Sehstörungen, Augen- \noder Muskelkontraktionen, Bewußtseinsverlust, Orientierungschwierigkeiten, \nunkontrollierte Bewegungen oder krampfartige Zuckungen.\n\n Wenn Sie erneut auf den Play-Button drücken, startet Die Funktion"
        
        self.warnLabel = customtkinter.CTkLabel(
            master=self,
            text="EPILEPSIE WARNUNG",
            height=22,
            text_color=parent.uiColor,
            bg_color="transparent")
        self.warnLabel.cget("font").configure(size=32, weight="bold")

       
        self.textBox = customtkinter.CTkTextbox(self,width=620, height=750, border_spacing=20)
        self.textBox.cget("font").configure(size=16)
        self.textBox.insert("0.0", self.text)  # insert at line 0 character 0
        self.textBox.configure(state="disabled")  # configure textbox to be read-only

        self.okay = customtkinter.CTkButton(
            self, 
            text="OK",
            fg_color="#555555",
            hover_color="#e24920",
            command=self.okCallback)
        self.okay.configure(state="disabled")

        self.cancel = customtkinter.CTkButton(
            self,
            text="Back",
            fg_color="#E86E4D",
            hover_color="#e24920",
            command=self.cancelCallback)
        
        self.checkbox = customtkinter.CTkCheckBox(
            master=self,
            text="Ich bin mir des Risikos bewusst.", 
            command=self.checkBoxCallback,
            onvalue="on", 
            offvalue="off",
            hover_color=parent.uiColor,
            border_color=parent.uiColor,
            fg_color=parent.hoverColor)
        self.checkbox.deselect()
        
        self.checkbox.place(relx=0.5,rely=0.7, anchor=customtkinter.S)
        self.warnLabel.place(relx=.5,rely=.05,anchor=customtkinter.N)
        self.textBox.place(relx=0.5,rely=0.5,anchor=customtkinter.CENTER)
        self.cancel.place(relx=0.6,rely=0.8, anchor=customtkinter.S)
        self.okay.place(relx=0.4,rely=0.8, anchor=customtkinter.S)

    def checkBoxCallback(self):
        if(self.checkbox.get() == "on"):
            self.okay.configure(state="normal",fg_color=self.parent.uiColor,hover_color=self.parent.hoverColor)
        else:
            self.okay.configure(state="disabled",fg_color="#555555")
    
    def okCallback(self):
        self.parent.warningNoticed = True
        self.parent.gradientSwitch.configure(state="normal", button_color=self.parent.uiColor, button_hover_color=self.parent.hoverColor, progress_color="#f8d3c9")
        self.parent.time_slider.get("Slider").configure(state="normal", button_color=self.parent.uiColor, button_hover_color=self.parent.hoverColor, progress_color="#f8d3c9")
        self.parent.deiconify()
        self.destroy()
        
    def cancelCallback(self):
        self.parent.deiconify()
        self.destroy()

In [14]:

class App(customtkinter.CTk):
    def __init__(self):
        super().__init__()
        #App Settings
        self.open = True
        self.speed = 1
        self.running = False
        self.warningNoticed = False
        self.switch_var = False

        self.hsv= [13,84,64]
        self.color = "#A3381B"
        self.bgSat = 0
        self.bgColor = "#000000"
        self.uiColor = "#E86E4D"
        self.hoverColor = "#e24920"
        self.borderColor = "#372e29"

        self.WarningWindow = None
        self.InfoWindow = None
        self.authors = "Henry Lorenz & Fabian Lampe"

        self.title("MMT Project")
        self.attributes('-fullscreen',True)
        self.config(bg = self.bgColor)
        
        #Elements
        # closeButton
        self.closeButton = customtkinter.CTkButton(
            master=self,
            text="✕",
            width=20,
            height=20, 
            command=self.closeCallback,
            fg_color=self.uiColor,
            hover_color=self.hoverColor)
        # / closeButton

        # Orange Box
        self.orangeBox = customtkinter.CTkFrame(
            master=self,
            width=200,
            height=200,
            corner_radius=50,
            bg_color=self.bgColor,
            fg_color=self.color)
        # /Orange Box

        # Authorlabel
        self.authorsLabel = customtkinter.CTkLabel(
            master=self,
            text= " Authors: " + str(self.authors),
            height=20,
            bg_color=self.bgColor,
            text_color="gray")
        self.authorsLabel.cget("font").configure(size=12)
        # /Authorlabel

        # settingsFrame
        self.settingsFrame = customtkinter.CTkFrame(
            master=self,
            width=350,
            height=330,
            border_width=1,
            bg_color=self.bgColor,
            border_color=self.borderColor)

        self.settingsLabel = customtkinter.CTkLabel(
            master=self.settingsFrame,
            text="Einstellungen ⚙",
            height=20,
            text_color="gray",
            bg_color="transparent")
        self.settingsLabel.cget("font").configure(size=16,weight="bold")

        # color Frame
        self.hsvFrame = customtkinter.CTkFrame(
            master=self.settingsFrame,
            width=335,
            height=150,
            fg_color="transparent",
            border_width=1,
            border_color=self.borderColor)

        self.hsvLabel = customtkinter.CTkLabel(
            master=self.hsvFrame,
            text="Farbe",
            height=22,
            text_color="gray",
            bg_color="transparent")
        self.hsvLabel.cget("font").configure(size=14, slant="italic")

        self.hexLabel = customtkinter.CTkLabel(
            master=self.hsvFrame,
            text=str(self.rgb2hex(self.hsv2rgb(self.hsv))),
            height=22,
            text_color=self.uiColor,
            bg_color="transparent")
        self.hexLabel.cget("font").configure(size=16, weight="bold")
        self.hexLabel.bind("<Enter>", lambda x: self.ToolTip.configure(text="HEX Farbcode der Farbbox"))
        self.hexLabel.bind("<Leave>", self.on_leave)
        #/ colorfrane

        # speed Frame
        self.speedFrame = customtkinter.CTkFrame(
            master=self.settingsFrame,
            width=335,
            height=100,
            fg_color="transparent",
            border_width=1,
            border_color=self.borderColor)
        
        self.speedLabel = customtkinter.CTkLabel(
            master=self.speedFrame,
            text="Zeit",
            height=22,
            text_color="gray",
            bg_color="transparent")
        self.speedLabel.cget("font").configure(size=14, slant="italic")
        # /speedframe

        #Tooltip
        self.ToolTip = customtkinter.CTkLabel(
            master=self.settingsFrame,
            text="",
            width=340,
            height=40,
            bg_color="transparent",
            text_color="gray")
        self.ToolTip.place(rely=1,relx=.5, anchor=customtkinter.S)
        self.ToolTip.cget("font").configure(size=12)
        #/Tooltip

        # color Inputs
        self.h_slider = self.buildSlider(self.hsvFrame, "Hue", 0, 360, self.hsv[0],self.s_sliderCallback, 
        lambda x: self.ToolTip.configure(text="Konstanter Hue Wert der Farbe (hsv)"), enabled=False)
        self.h_slider.get("Slider").destroy()

        self.s_slider = self.buildSlider(self.hsvFrame, "Saturation", 30, 100, self.hsv[1], self.s_sliderCallback,
        lambda x: self.ToolTip.configure(text="Sättigungs Wert der Farbe (hsv)"))

        self.v_slider = self.buildSlider(self.hsvFrame, "Value", 10, 100, self.hsv[2], self.v_sliderCallback,
        lambda x: self.ToolTip.configure(text="Helligkeit der Farbe (hsv)"))

        self.bg_slider = self.buildSlider(self.hsvFrame, "BG Saturation", 0, 100, self.bgSat, self.bg_sliderCallback,
        lambda x: self.ToolTip.configure(text="Helligkeit der Hintergrundfarbe"))

        self.hexLabel.place(relx=0.95,rely=0.1,anchor=customtkinter.NE)
        self.h_slider.get("Frame").place(relx=.005, rely=.35, anchor=customtkinter.SW)
        self.s_slider.get("Frame").place(relx=.005, rely=.575, anchor=customtkinter.SW)
        self.v_slider.get("Frame").place(relx=.005, rely=.775, anchor=customtkinter.SW)
        self.bg_slider.get("Frame").place(relx=.005, rely=.975, anchor=customtkinter.SW)

        # /color inputs

        self.gradientSwitch = customtkinter.CTkSwitch(
            master=self.speedFrame, text="Gradient",
            command=self.gradientSwitchCallback,
            onvalue="on", 
            offvalue="off",
            button_color="#555555",
            progress_color="#f8d3c9",
            text_color="gray",
            state="disabled",
            button_hover_color=self.hoverColor)
        self.gradientSwitch.bind("<Enter>", lambda x: self.ToolTip.configure(text="Hintergrundfarbe wechselt über Graustufen"))
        self.gradientSwitch.bind("<Leave>", self.on_leave)

        # time Inputs
        self.time_slider = self.buildSlider(self.speedFrame, "Geschwindigkeit", 0.5, 2, self.speed, self.time_sliderCallback, 
        lambda x: self.ToolTip.configure(text="Geschwindigkeit des Hintergrundfarbwechsels (sek)"), False)

        self.time_button =customtkinter.CTkButton(
            master= self.speedFrame,
            text="▶",
            width=90,
            height=35, 
            command=self.pauseCallback,
            fg_color=self.uiColor,
            hover_color=self.hoverColor
        )
        self.time_button.bind("<Enter>", lambda x: self.ToolTip.configure(text="Startet/Stoppt automatische Farbänderung \nder Hintergrundfarbe"))
        self.time_button.bind("<Leave>", self.on_leave)
 

        self.info_button =customtkinter.CTkButton(
            master= self.speedFrame,
            text="❓",
            width=35,
            height=35, 
            command=self.infoCallback,
            fg_color=self.uiColor,
            hover_color=self.hoverColor
        )
        self.info_button.cget("font").configure(size=20, weight="bold")
        self.info_button.bind("<Enter>", lambda x: self.ToolTip.configure(text="Informationen zur Illusion und zum Program"))
        self.info_button.bind("<Leave>", self.on_leave)

        self.time_slider.get("Frame").place(relx=.005, rely=.45, anchor=customtkinter.SW)
        
        self.time_button.place(relx=.34,rely=0.92,anchor=customtkinter.SE)
        self.info_button.place(relx=.96,rely=0.92,anchor=customtkinter.SE)
        self.gradientSwitch.place(relx=0.39,rely=0.85,anchor=customtkinter.SW)
        # /time Inputs

        # /settingsFrame

        #Place Elements
        #general Elements
        self.closeButton.place(relx = 1, rely = 0,anchor=customtkinter.NE)
        self.closeButton.bind("<Enter>", 
        lambda x: self.ToolTip.configure(text="Beendet das Programm"))
        self.closeButton.bind("<Leave>", self.on_leave)

        self.authorsLabel.place(relx = 0, rely = 1, anchor=customtkinter.SW)
        self.orangeBox.place(relx=0.5, rely=0.5, anchor=customtkinter.CENTER)

        #hsvFrame Elements
        self.hsvFrame.place(relx=.02,rely=.1,anchor=customtkinter.NW)
        self.hsvLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)

        self.speedFrame.place(relx=.02,rely=.88,anchor=customtkinter.SW)
        self.speedLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)


        #Setting Elements
        self.settingsFrame.place(relx=1, rely=1, anchor=customtkinter.SE)
        self.settingsLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)

# Callback Methodes

    def on_leave(self, enter):
        self.ToolTip.configure(text="")

    def closeCallback(self):
        self.open = False
        self.destroy()
    
    def gradientSwitchCallback(self):
        self.switch_var = not self.switch_var

    def s_sliderCallback(self,value):
        self.hsv[1] = value
        self.s_slider.get("Value").configure(text="{0:>2}".format(str(round(self.hsv[1]))))
        self.color=str(self.rgb2hex(self.hsv2rgb(self.hsv)))
        self.hexLabel.configure(text="{}".format(self.color))
        self.orangeBox.configure(fg_color=self.color)
    
    def v_sliderCallback(self,value):
        self.hsv[2] = value
        self.v_slider.get("Value").configure(text="{0:>2}".format(str(round(self.hsv[2]))))
        self.color=str(self.rgb2hex(self.hsv2rgb(self.hsv)))
        self.hexLabel.configure(text="{}".format(self.color))
        self.orangeBox.configure(fg_color=self.color)

    def time_sliderCallback(self,value):
        self.speed = value
        self.time_slider.get("Value").configure(text="{0:>2}".format(str(round(self.speed,1))))

    def bg_sliderCallback(self,value):
        self.bgSat = value
        self.bg_slider.get("Value").configure(text="{0:>2}".format(str(round(self.bgSat))))
        self.bgColor = self.rgb2hex(self.hsv2rgb([0,0,self.bgSat]))
        self.config(bg=self.bgColor)
    
    def pauseCallback(self):
        if(self.warningNoticed == False): self.create_toplevel()
        else:
            if(self.running):
                self.running = False
                self.time_button.configure(text="▶")
                self.bg_slider.get("Slider").configure(state = "normal", button_color=self.uiColor, button_hover_color=self.hoverColor, progress_color="#f8d3c9")
            else:
                self.running = True
                self.time_button.configure(text="⏸")
                self.bg_slider.get("Slider").configure(state ="disabled", button_color="#555555",progress_color="#999999")  


# Layout Methodes
    def buildSlider(self, origin, name, start, end, value, callback, tooltip=None, enabled=True):
        sliderFrame = customtkinter.CTkFrame(
            master=origin,
            width=330,
            height=20,
            fg_color="transparent",
            border_width=0,
            border_color=self.borderColor)

        if(enabled): Slider = customtkinter.CTkSlider(
                master = sliderFrame,
                from_=start, 
                to=end,
                button_color=self.uiColor,
                button_hover_color=self.hoverColor,
                progress_color="#f8d3c9",
                hover=True,
                command=callback)  
        else: Slider = customtkinter.CTkSlider(
                master = sliderFrame,
                from_=start, 
                to=end,
                button_color="#555555",
                progress_color="#999999",
                state="disabled",
                command=callback)
        Slider.set(value)

        Label = customtkinter.CTkLabel(
            master= sliderFrame,
            text=name,
            height=20,
            text_color="gray",
            bg_color="transparent")
        Label.cget("font").configure(size=12)
        
        Value = customtkinter.CTkLabel(
            master=sliderFrame,
            text= str(value),
            height=20,
            text_color="gray",
            bg_color="transparent")
        Value.cget("font").configure(size=12)
        
        Label.place(relx=.275, rely=.5, anchor=customtkinter.E)
        Value.place(relx=.355, rely=.5, anchor=customtkinter.E)
        Slider.place(relx=.38, rely=.5, anchor=customtkinter.W)

        sliderFrame.bind("<Enter>", tooltip)
        Label.bind("<Enter>", tooltip)
        Value.bind("<Enter>", tooltip)
        Slider.bind("<Enter>", tooltip)

        sliderFrame.bind("<Leave>", self.on_leave)
        Label.bind("<Leave>", self.on_leave)
        Value.bind("<Leave>", self.on_leave)
        Slider.bind("<Leave>", self.on_leave)

        return {"Frame":sliderFrame,
                "Label": Label,
                "Value":Value,
                "Slider":Slider}
            


    def create_toplevel(self):
       self.WarningWindow = WarningWindow(self)
       self.withdraw()
    
    def infoCallback(self):
        self.InfoWindow = InfoWindow(self)
        self.withdraw()
       
# Utility Methods
    def rgb2hex(self,rgb):
            r,g,b = round(rgb[0]),round(rgb[1]),round(rgb[2])
            val = "#{:02X}{:02X}{:02X}".format(r,g,b)
            return val

    def hex2rgb(self,hex):
            h = hex.strip("#") 
            rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
            return rgb
        
    def rgb2hsv(self,rgb):
            r,g,b = (rgb[0]/255),(rgb[1]/255),(rgb[2]/255)
            # Constraining the values to the range 0 to 1
            # defining the following terms for convenience
            Cmax = max(r, g, b)
            Cmin = min(r, g, b)
            delta = Cmax - Cmin
            # hue calculation
            if (delta == 0):  H = 0
            elif (Cmax == r): H = (60 * (((g  - b) / delta) % 6))
            elif (Cmax == g): H = (60 * (((b  - r) / delta) + 2))
            elif (Cmax == b): H = (60 * (((r  - g) / delta) + 4))
            
            # saturation calculation
            if (Cmax == 0): S = 0 
            else: S = delta / Cmax
            # value calculation
            V = Cmax 
            return(round(H),round(S*100),round(V*100))

    def hsv2rgb(self,hsv):
            h,s,v = (hsv[0]/360),(hsv[1]/100),(hsv[2]/100)
            rgb= colorsys.hsv_to_rgb(h,s,v)
            return (round(rgb[0]*255),round(rgb[1]*255),round(rgb[2]*255))

In [15]:
if __name__ == "__main__":
    app = App()
    time_1 = 0
    switch=True

    while 1:
        time_2 = time.process_time()
        app.update_idletasks()
        app.update()
        delta = time_2 - time_1        
 
        if app.running and app.switch_var:
            if(delta >= app.speed):switch = not switch
            if switch and delta<app.speed:
                    app.bgSat=100-round((delta*100)/app.speed)
                    app.bgColor = app.rgb2hex(app.hsv2rgb([0,0,app.bgSat]))
                    app.config(bg=app.bgColor)
                    app.bg_slider.get("Slider").set(app.bgSat)
                    app.bg_slider.get("Value").configure(text="{0:>2}".format(str(round(app.bgSat))))
            elif (not switch) and (delta<app.speed):
                    app.bgSat=round((delta*100)/app.speed)
                    app.bgColor = app.rgb2hex(app.hsv2rgb([0,0,app.bgSat]))
                    app.config(bg=app.bgColor)
                    app.bg_slider.get("Slider").set(app.bgSat)
                    app.bg_slider.get("Value").configure(text="{0:>2}".format(str(round(app.bgSat))))
            
            
        elif app.running and not app.switch_var and delta > app.speed:
            if app.cget("background") == "black":
                app.config(bg = "white")
                app.orangeBox.configure(bg_color="white")
            else:
                app.config(bg = "black")
                app.orangeBox.configure(bg_color="black")
        
        if delta > app.speed: time_1 = time_2
        if(app.open == False):
            break

invalid command name "139734657368000_click_animation"
    while executing
"139734657368000_click_animation"
    ("after" script)
